In [370]:
#габариты и координаты (в см)
from typing import Union

class Coords:
    def __init__(self, x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):  
        return "(x:% s y:% s z:% s)" % (self.x, self.y, self.z)  #подумать над разделением

In [371]:
d = Coords("f", 2, 3)
print(d)

(x:f y:2 z:3)


In [372]:
#размеры и расположение объектов

#координаты: середина дна гарнитура 
#габариты: x - длина, y - высота, z - ширина 

class Kitchen_Object:
    def __init__(self, size: Coords, coords: Coords):
        self.size = size
        self.coords = coords
    
    #метод нахождения координат двух углов проекции (дна) гарнитура (на одной из диагонали)
    @classmethod
    def corner_coords(self, size, coords):
        x = [coords.x - size.x/2, coords.z + size.z/2]
        z = [coords.x + size.x/2, coords.z - size.z/2]
        return x, z
    
    #координаты середины сторон гарнитура (без высоты)
#     @classmethod
#     def middle_coords(self, size, coords):
#         a = [coords.x - size.x/2, coords.z]
#         b = [coords.x, coords.z + size.z/2]
#         c = [coords.x + size.x/2, coords.z]
#         d = [coords.x, coords.z - size.z/2]
#         return [a, b, c, d]
    
    @classmethod
    def _bottom_overlap(cls, size1, size2, coords1, coords2):
        coords1 = Kitchen_Object.corner_coords(size1, coords1) # координаты по обеим концам диагонали
        top1, bottom1 = coords1[0], coords1[1] # координаты "верхней" и "нижней" точки проекции гарнитура(x;z)
        coords2 = Kitchen_Object.corner_coords(size2, coords2)
        top2, bottom2 = coords2[0], coords2[1]
        #print(top1, bottom1, top2, bottom2)
        if top1[0] == bottom1[0] or top1[1] == bottom1[1] or top2[0] == bottom2[0] or top2[1] == bottom2[1]: #площадь 0
            return False
        if top1[0] > bottom2[0] or top2[0] >  bottom1[0]: #слева/справа
            return False
        if bottom1[1] > top2[1] or bottom2[1] > top1[1]: #сверху/снизу 
            return False
        return True
    
    def _height_overlap(self, size1, size2, coords1, coords2):
        height1 = size1.y
        height2 = size2.y
        y1 = coords1.y
        y2 = coords2.y
        if y1 + height1 < y2:
            return False
        if y2 + height2 < y1:
            return False 
        return True
    
    @classmethod
    def overlap(self, object1, object2):
        if self._bottom_overlap(object1.size, object2.size, object1.coords, object2.coords) == False:
            return False
        if self._height_overlap(self, object1.size, object2.size, object1.coords, object2.coords):
            return True

In [373]:
k = Kitchen_Object((2,4,6), (3,4,5))
print(k)

In [374]:
from typing import Union

#пусть координаты - середина дна кухонного шкафа

class Cabinet(Kitchen_Object):
    def __init__(self, material, size, coords):
        super().__init__(size, coords)
        self.material = material
        
#     def __repr__(self):  
#         return "Cabinet(material: % s, size: % s, coords: % s)" % (self.material, self.size, self.coords) 
    def __repr__(self):  
        return "Cabinet(material: % s)" % self.material

    @property
    def material(self):
        return self.__material
    
    @material.setter 
    def material(self, material: Union[str, int]):
        self.__material = material
        
#TODO: возможно как-то переопределить (для красоты)
#     @classmethod
#     def overlap(self, cabinet1, cabinet2):

In [375]:
cab1 = Cabinet("wood", Coords(3,2,4), Coords(3,5,6))
cab2 = Cabinet("wood", Coords(3,2,4), Coords(3,5,6))
cab1.size.z
print(Kitchen_Object.overlap(cab1, cab2))

True


In [376]:
class Appliance(Kitchen_Object):
    def __init__(self, name, size, coords):
        super().__init__(size, coords)
        self.__name = name

    @property
    def name(self):
        return self.__name
    
    @name.setter 
    def name(self, name: Union[str, int]):
        self.__name = name
        
#     def __repr__(self):  
#         return "Appliance(name: % s, size: % s, coords: % s)" % (self.name, self.size, self.coords) 

    def __repr__(self):  
        return "Appliance(name: % s)" % self.name


        
    #не поняла, зачем тут метод включения/выключения техники (???)

In [377]:
app1 = Appliance("TV", Coords(3,2,4), Coords(3,5,6))
app2 = Appliance("Oven", Coords(3,2,4), Coords(3,5,6))
a = 1
print(Appliance.overlap(cab1, cab2))
type(app1)
type(a)

True


int

In [378]:
isinstance(app1, Appliance)

True

In [379]:
from collections import namedtuple
import math

kitchen_size = namedtuple('KitchenSize', ['x', 'y'])

class Kitchen:
    def __init__(self, size: kitchen_size, content: list):
        self.__size = size
        self.content = content 
        
    @property
    def size(self):
        return self.__size
    
    @size.setter 
    def size(self, size: kitchen_size):
        self.__size = size
     
    #def __find_appliance(self, content: list):
    def find_appliance(self, content: list):
        appliances = []
        for x in content:
            if isinstance(x, Appliance):
                appliances.append(x)
        return appliances
    
    #def __find_cabinet(self, content):
    def find_cabinet(self, content):
        cabinets = []
        for x in content:
            if isinstance(x, Cabinet):
                cabinets.append(x)
        return cabinets
    
# Метод для проверки, не висит ли техника в воздухе:
#     - если координата по высоте = 0, то не висит
#     - если больше нуля, то проверяем, не стоит ли техника на шкафе
#     - пока будем считать, что техника не может висеть на стене (но есть заготовка см. класс Kitchen_Object)

    def appliance_float(self):
        res = []
        appliance = self.find_appliance(self.content)
        cabinets = self.find_cabinet(self.content)
        for app in appliance:
            if app.coords.y == 0:
                res.append("%s doesn't float" % app.name)
            else:
                counter = 0
                for cab in cabinets:
                    if Kitchen_Object._bottom_overlap(cab.size, app.size, cab.coords, app.coords) and \
                    app.coords.y == cab.size.y + cab.coords.y or app.coords.x == 0 or app.coords.z == 0:
                        counter = counter + 1
                if counter >= 1:
                    res.append("%s doesn't float" % app.name)
                else:
                    res.append("%s floats" % app.name)
        return res 
    
    def same_material_cabinets_too_close(self):
        res = set()
        cabinets = self.find_cabinet(self.content)
        for i in range(0, len(cabinets), 1):
            for j in range(0, len(cabinets), 1):
                if cabinets[i].material == cabinets[j].material and i != j \
                and math.dist([cabinets[i].coords.x, cabinets[i].coords.z], [cabinets[j].coords.x, cabinets[j].coords.z]) \
                < 200:
                    res.add(cabinets[i].material)
        for x in res:
            print("Cabinets of %s are too close!" % x)
            
    def overlap(self):
        counter = 0 
        res = set()
        for x in self.content:
            for y in self.content:
                if x != y and Kitchen_Object.overlap(x, y) == True:
                    new_pair = (x, y)
                    if (y, x) not in res:
                        res.add(new_pair)
        return res


In [380]:
cabinet1 = Cabinet("oak", Coords(1, 3, 2), Coords(0.5, 0, 3))
cabinet2 = Cabinet("oak", Coords(2, 3, 1), Coords(2, 0, 0.5))
appl1 = Appliance("TV", Coords(1, 1, 1), Coords(2.5, 3, 0.5))
appl2 = Appliance("Oven", Coords(1, 1, 1), Coords(4.5, 3, 2.5))
appl3 = Appliance("Oven2", Coords(1, 1, 1), Coords(3, 0, 4.5))
content = [cabinet1, cabinet2, appl1, appl2, appl3]
size = kitchen_size(100, 100)
k = Kitchen(size, content)
print(*k.appliance_float())
k.same_material_cabinets_too_close()
print("Pieces of furniture that overlap:", k.overlap())

TV doesn't float Oven floats Oven2 doesn't float
Cabinets of oak are too close!
Pieces of furniture that overlap: {(Cabinet(material: oak), Appliance(name: TV))}
